## BME 544 Assignment 3

In [ ]:
import os
import librosa
import librosa.display 
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import pandas as pd
from scipy.signal import hilbert, freqz, zpk2tf, butter, lfilter

np.random.seed(42)

### 1. Examine Recordings

In [ ]:
def read_timeseries(file_path):
    df = pd.read_csv(file_path, sep='\t', skiprows=1, names=["Sample_Index", "Signal_mV"])
    df["Time_s"] = df["Sample_Index"] / 1000
    return df

def plot_timeseries_slice(df, start_pct, end_pct, title=""):
    total_samples = len(df)
    start_idx = int(total_samples * start_pct / 100)
    end_idx = int(total_samples * end_pct / 100)
    slice_df = df.iloc[start_idx:end_idx]
    
    plt.figure(figsize=(10, 5))
    plt.plot(slice_df["Time_s"], slice_df["Signal_mV"])
    plt.xlabel("Time [s]")
    plt.ylabel("Signal [mV]")
    plt.title(title)
    plt.show()

def welch_psd(signal, fs=1000, segment_length=1024, overlap=128):
    segments = []
    for start in range(0, len(signal) - segment_length + 1, segment_length - overlap):
        segment = signal[start:start + segment_length]
        segments.append(segment)

    segments = np.array(segments)
    
    psd = []
    for segment in segments:
        dft = np.fft.fft(segment)
        positive_dft = dft[:segment_length // 2]
        magnitude_squared = np.abs(positive_dft) ** 2
        psd.append(magnitude_squared)
    
    psd_avg = np.mean(psd, axis=0)

    f = np.fft.fftfreq(segment_length, d=1/fs)[:segment_length // 2]
    
    psd_avg /= (segment_length * fs / 2)
    
    return f, psd_avg

file_path = "DC_2/processed/A87FK0_Clean.txt"
data = read_timeseries(file_path)
plot_timeseries_slice(data, 30, 40, "Clean")
file_path = "DC_2/processed/A87FK0_NoFilter.txt"
data = read_timeseries(file_path)
plot_timeseries_slice(data, 30, 40, "NoFilter")
file_path = "DC_2/processed/A87FK0_EMG.txt"
data = read_timeseries(file_path)
plot_timeseries_slice(data, 55, 65, "EMG")

In [ ]:
def plot_psd(df, label):
    f, psd = welch_psd(df["Signal_mV"].values)
    
    freq_resolution = f[1] - f[0]
    print(f"Frequency resolution: {freq_resolution:.2f} Hz")
    
    plt.semilogy(f[f <= 150], psd[f <= 150], label=label)

plt.figure(figsize=(10, 5))

file_path = "DC_2/processed/A87FK0_Clean.txt"
data = read_timeseries(file_path)
plot_psd(data, "Clean")

file_path = "DC_2/processed/A87FK0_NoFilter.txt"
data = read_timeseries(file_path)
plot_psd(data, "NoFilter")

file_path = "DC_2/processed/A87FK0_EMG.txt"
data = read_timeseries(file_path)
plot_psd(data, "EMG")

plt.xlabel("Frequency [Hz]")
plt.ylabel("Power Spectral Density")
plt.title("Power Spectral Density (PSD) [0-150 Hz]")
plt.legend()
plt.show()

### 2. Time-synchronous Averaging

In [ ]:
def read_timeseries(file_path):
    df = pd.read_csv(file_path, sep='\t', skiprows=1, names=["Sample_Index", "Signal_mV"])
    return df

def read_peaks(file_path):
    df = pd.read_csv(file_path, sep='\t', skiprows=1, header=None, names=["Sample_Index"])
    return df

def plot_timeseries_slice(df, start_pct, end_pct, peaks, title=""):
    total_samples = len(df)
    start_idx = int(total_samples * start_pct / 100)
    end_idx = int(total_samples * end_pct / 100)
    slice_df = df.iloc[start_idx:end_idx]
    peaks = peaks[(peaks["Sample_Index"] >= start_idx) & (peaks["Sample_Index"] < end_idx)]
    plt.figure(figsize=(13, 6))
    plt.plot(slice_df["Sample_Index"], slice_df["Signal_mV"])
    plt.scatter(peaks["Sample_Index"], df.loc[peaks["Sample_Index"], "Signal_mV"], color='red', label="Peaks")
    plt.axvline(x = 9833, color = 'b', label = 'axvline - full height')
    plt.axvline(x = 10422, color = 'b', label = 'axvline - full height')
    plt.axvline(x = 10773, color = 'b', label = 'axvline - full height')
    plt.axvline(x = 11292, color = 'b', label = 'axvline - full height')
    midpoint_c1 = (10422 + 9833) // 2
    midpoint_c2 = (11292 + 10773) // 2

    plt.axvline(x = midpoint_c1, color = 'g', label = 'axvline - full height')
    plt.axvline(x = midpoint_c2, color = 'g', label = 'axvline - full height')

    plt.xlabel("Sample #")
    plt.ylabel("Signal [mV]")
    plt.title(title)
    plt.show()

def extract_segments(signal_df, peaks_df, n1, n2):
    segments = []
    total_samples = len(signal_df)

    for peak in peaks_df["Sample_Index"]:
        start_idx = max(0, peak - n1)
        end_idx = min(total_samples, peak + n2 + 1)
        segment = signal_df.iloc[start_idx:end_idx].copy()
        if len(segment) == 906:
            segments.append(np.array(segment["Signal_mV"]))

    return segments

clean_data_file_path = "DC_2/processed/A87FK0_Clean.txt" 
clean_peaks_file_path = "DC_2/processed/A87FK0_Clean_Peaks.txt"  
clean_data = read_timeseries(clean_data_file_path)
clean_peaks = read_peaks(clean_peaks_file_path)
plot_timeseries_slice(clean_data, 30, 40, clean_peaks, "Clean")

midpoint_c1 = (10422 + 9833) // 2
midpoint_c2 = (11292 + 10773) // 2
n1 = abs(midpoint_c1 - 10550)
n2 = abs(midpoint_c2 - 10550)

clean_sliced_signal = np.array(clean_data["Signal_mV"])[midpoint_c1:midpoint_c2]
clean_segments = extract_segments(clean_data, clean_peaks, n1, n2)
avg_segment = np.mean(clean_segments, axis=0)

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

axes[0].plot(np.arange(len(clean_sliced_signal)) / 1000, clean_sliced_signal)
axes[0].set_title("Single cardiac cycle (Clean)")
axes[0].set_xlabel("Time [s]")
axes[0].set_ylabel("Signal [mV]")

axes[1].plot(np.arange(len(avg_segment)) / 1000, avg_segment)
axes[1].set_title("Time-synchronous averaged cardiac cycle (Clean)")
axes[1].set_xlabel("Time [s]")
axes[1].set_ylabel("Signal [mV]")

plt.tight_layout()
plt.show()

In [ ]:
no_filter_data_file_path = "DC_2/processed/A87FK0_NoFilter.txt" 
no_filter_peaks_file_path = "DC_2/processed/A87FK0_NoFilter_Peaks.txt"  
no_filter_data = read_timeseries(no_filter_data_file_path)
no_filter_peaks = read_peaks(no_filter_peaks_file_path)

chosen_peak_ind = 9637

no_filter_sliced_signal = np.array(no_filter_data["Signal_mV"])[chosen_peak_ind - n1:chosen_peak_ind + n2 + 1]
no_filter_segments = extract_segments(no_filter_data, no_filter_peaks, n1, n2)
no_filter_avg_segment = np.mean(no_filter_segments, axis=0)

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

axes[0].plot(np.arange(len(no_filter_sliced_signal)) / 1000, no_filter_sliced_signal)
axes[0].set_title("Single cardiac cycle (NoFilter)")
axes[0].set_xlabel("Time [s]")
axes[0].set_ylabel("Signal [mV]")

axes[1].plot(np.arange(len(no_filter_avg_segment)) / 1000, no_filter_avg_segment)
axes[1].set_title("Time-synchronous averaged cardiac cycle (NoFilter)")
axes[1].set_xlabel("Time [s]")
axes[1].set_ylabel("Signal [mV]")

plt.tight_layout()
plt.show()

In [ ]:
emg_data_file_path = "DC_2/processed/A87FK0_EMG.txt" 
emg_peaks_file_path = "DC_2/processed/A87FK0_EMG_Peaks.txt"  
emg_data = read_timeseries(emg_data_file_path)
emg_peaks = read_peaks(emg_peaks_file_path)

chosen_peak_ind = 18092

emg_sliced_signal = np.array(emg_data["Signal_mV"])[chosen_peak_ind - n1:chosen_peak_ind + n2 + 1]
emg_segments = extract_segments(emg_data, emg_peaks, n1, n2)
emg_avg_segment = np.mean(emg_segments, axis=0)

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

axes[0].plot(np.arange(len(emg_sliced_signal)) / 1000, emg_sliced_signal)
axes[0].set_title("Single cardiac cycle (EMG)")
axes[0].set_xlabel("Time [s]")
axes[0].set_ylabel("Signal [mV]")

axes[1].plot(np.arange(len(emg_avg_segment)) / 1000, emg_avg_segment)
axes[1].set_title("Time-synchronous averaged cardiac cycle (EMG)")
axes[1].set_xlabel("Time [s]")
axes[1].set_ylabel("Signal [mV]")

plt.tight_layout()
plt.show()

### 3. Notch Filter

In [ ]:
fs = 1000
f0 = 60
omega0 = 2 * np.pi * f0 / fs

z = [np.exp(1j * omega0), np.exp(-1j * omega0)]

r1 = 0
p1 = []

b1, a1 = zpk2tf(z, p1, 1)
w, H1 = freqz(b1, a1, 2*fs, fs)

r2 = 0.8
p2 = [r2 * np.exp(1j * omega0), r2 * np.exp(-1j * omega0)]
b2, a2 = zpk2tf(z, p2, 1)
w, H2 = freqz(b2, a2, 2*fs, fs)

r3 = 0.98
p3 = [r3 * np.exp(1j * omega0), r3 * np.exp(-1j * omega0)]
b3, a3 = zpk2tf(z, p3, 1)
w, H3 = freqz(b3, a3, 2*fs, fs)

plt.figure()
plt.plot(w * fs / (2 * np.pi), 20 * np.log10(np.abs(H1)), 'b', label='Zeros only', linewidth=1)
plt.plot(w * fs / (2 * np.pi), 20 * np.log10(np.abs(H2)), 'r', label='Poles and Zeros (r=0.8)', linewidth=1)
plt.plot(w * fs / (2 * np.pi), 20 * np.log10(np.abs(H3)), 'g', label='Poles and Zeros (r=0.98)', linewidth=1)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude (dB)')
plt.title('Magnitude Response of Notch Filters')
plt.legend()    
plt.xlim([0, fs // 2])
plt.ylim([-300, 20])
plt.grid(True)
plt.show()

In [ ]:
clean_sliced_signal_notch_filtered = lfilter(b3, a3, clean_sliced_signal)

plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 2)
plt.plot(np.arange(len(clean_sliced_signal)) / 1000, clean_sliced_signal, label='Unfiltered Signal (r=0.98)', color='b', linewidth=1.5)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(np.arange(len(clean_sliced_signal)) / 1000, clean_sliced_signal_notch_filtered, label='Filtered Signal (r=0.98)', color='g', linewidth=1.5)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.legend()

plt.tight_layout()
plt.show()

### 4. Low-pass Filter

In [ ]:
cutoff = 20
sampling_rate = 1000
order = 4

nyquist = sampling_rate / 2
normal_cutoff = cutoff / nyquist
b, a = butter(order, normal_cutoff, btype='lowpass')

no_filter_data_lowpass_filtered = lfilter(b3, a3, np.array(no_filter_data["Signal_mV"]))
no_filter_data_lowpass_notch_filtered = lfilter(b, a, no_filter_data_lowpass_filtered)

def plot_psd(_signal, label):
    f, psd = welch_psd(_signal)
    
    plt.semilogy(f[f <= 150], psd[f <= 150], label=label, linewidth=1)

plt.figure(figsize=(10, 5))

plot_psd(no_filter_data_lowpass_notch_filtered, "Filtered")
plot_psd(np.array(clean_data["Signal_mV"]), "Clean")

plt.xlabel("Frequency [Hz]")
plt.ylabel("Power Spectral Density")
plt.title("Clean vs Filtered Power Spectral Density (PSD)[0-150 Hz]")
plt.legend()
plt.show()

In [ ]:
chosen_peak_ind = 9637

no_filter_data_lowpass_notch_filtered_sliced = no_filter_data_lowpass_notch_filtered[chosen_peak_ind - n1:chosen_peak_ind + n2 + 1]

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

axes[0].plot(np.arange(len(no_filter_data_lowpass_notch_filtered_sliced)) / 1000, no_filter_data_lowpass_notch_filtered_sliced)
axes[0].set_title("Filtered cardiac cycle (NoFilter)")
axes[0].set_xlabel("Time [s]")
axes[0].set_ylabel("Signal [mV]")

axes[1].plot(np.arange(len(clean_sliced_signal)) / 1000, clean_sliced_signal)
axes[1].set_title("Single cardiac cycle (Clean)")
axes[1].set_xlabel("Time [s]")
axes[1].set_ylabel("Signal [mV]")

plt.tight_layout()
plt.show()